# **Step 1: Initial Setup and Loading the Dataset**

In this step,

* Import the necessary libraries

* Check if the dataset is uploaded

* Load the CSV file

* Display the first few rows to see what the data looks like

In [1]:
# Step 1: Initial Setup and Data Loading

# importing basic libraries
import pandas as pd
import numpy as np

# checking what files are uploaded in this Colab session
import os
print("Files in the current working directory:")
print(os.listdir())  # this lists the uploaded files

# reading the dataset from the uploaded CSV file
# update the file name here if yours is different
df = pd.read_csv('/content/dataset.csv')

# looking at the first few rows to understand the structure
print("\nPreview of the dataset:")
df.head()

Files in the current working directory:
['.config', 'dataset.csv', 'sample_data']

Preview of the dataset:


,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00


In [29]:
df.info()  # Description

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18368 entries, 0 to 18367
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ID                      18368 non-null  int64  
 1   SystemCodeNumber        18368 non-null  object 
 2   Capacity                18368 non-null  int64  
 3   Latitude                18368 non-null  float64
 4   Longitude               18368 non-null  float64
 5   Occupancy               18368 non-null  int64  
 6   VehicleType             18368 non-null  object 
 7   TrafficConditionNearby  18368 non-null  object 
 8   QueueLength             18368 non-null  int64  
 9   IsSpecialDay            18368 non-null  int64  
 10  LastUpdatedDate         18368 non-null  object 
 11  LastUpdatedTime         18368 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 1.7+ MB


# **Step 2: Explore and Clean the Data**

In this step:

* Check the column names

* Look at data types

* Check for missing values

In [2]:
# Step 2: Exploring and Cleaning the Data

# checking the names of all columns
print("Columns in the dataset:")
print(df.columns)

# checking number of rows and columns
print("\nDataset shape (rows, columns):")
print(df.shape)

# checking what kind of data is in each column
print("\nData types of each column:")
print(df.dtypes)

# checking if there are any missing (NaN) values
print("\nMissing values in each column:")
print(df.isnull().sum())

# showing some random rows to better understand the data
print("\nSome random samples from the data:")
df.sample(5)

Columns in the dataset:
Index(['ID', 'SystemCodeNumber', 'Capacity', 'Latitude', 'Longitude',
       'Occupancy', 'VehicleType', 'TrafficConditionNearby', 'QueueLength',
       'IsSpecialDay', 'LastUpdatedDate', 'LastUpdatedTime'],
      dtype='object')

Dataset shape (rows, columns):
(18368, 12)

Data types of each column:
ID                          int64
SystemCodeNumber           object
Capacity                    int64
Latitude                  float64
Longitude                 float64
Occupancy                   int64
VehicleType                object
TrafficConditionNearby     object
QueueLength                 int64
IsSpecialDay                int64
LastUpdatedDate            object
LastUpdatedTime            object
dtype: object

Missing values in each column:
ID                        0
SystemCodeNumber          0
Capacity                  0
Latitude                  0
Longitude                 0
Occupancy                 0
VehicleType               0
TrafficConditionNearby  

,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime
12943,12943,Others-CCCPS135a,3883,26.147499,91.728005,832,bike,low,3,0,10-12-2016,08:29:00
14446,14446,Others-CCCPS8,1322,26.147549,91.727995,753,cycle,average,4,0,04-10-2016,14:57:00
5624,5624,BHMNCPHST01,1200,26.140014,91.731000,611,car,average,4,0,26-10-2016,15:59:00
6674,6674,BHMNCPNST01,485,26.140048,91.730972,323,bike,average,4,0,10-10-2016,10:57:00
17099,17099,Shopping,1920,26.150504,91.733531,1366,car,average,6,0,06-10-2016,11:30:00


# **Step 3: Model 1 — Baseline Linear Pricing**

In this model, created a simple rule:

* Price at time ``t+1 = Price at time t + α × (Occupancy / Capacity)``

and check:

* Set a base price (e.g., $10)

* Use a small α (like 5) to slowly increase price as parking fills up



In [3]:
# Step 3: Baseline Linear Pricing Model

# setting the base price for all parking lots
base_price = 10

# setting alpha value — controls how fast price increases with occupancy
alpha = 5  # you can change this later to tune the model

# function to calculate next price based on current occupancy and capacity
def calculate_linear_price(occupancy, capacity, prev_price):
    occ_ratio = occupancy / capacity
    new_price = prev_price + alpha * occ_ratio
    return round(new_price, 2)

# making a copy of the original DataFrame to add price column
df_model1 = df.copy()

# creating a column to hold the calculated price
# initially setting all to base price
df_model1['Price_Model1'] = float(base_price)  # forces float dtype

# now applying the pricing logic row by row
# assuming the rows are in time order for each parking lot
for i in range(1, len(df_model1)):
    if df_model1.loc[i, 'SystemCodeNumber'] == df_model1.loc[i - 1, 'SystemCodeNumber']:
        prev_price = df_model1.loc[i - 1, 'Price_Model1']
    else:
        prev_price = base_price  # reset price for new parking lot
    occ = df_model1.loc[i, 'Occupancy']
    cap = df_model1.loc[i, 'Capacity']
    df_model1.loc[i, 'Price_Model1'] = calculate_linear_price(occ, cap, prev_price)

# showing the first few rows with the new price column
print("Sample with baseline price:")
df_model1[['SystemCodeNumber', 'Occupancy', 'Capacity', 'Price_Model1']].head(10)



Sample with baseline price:


,SystemCodeNumber,Occupancy,Capacity,Price_Model1
0,BHMBCCMKT01,61,577,10.00
1,BHMBCCMKT01,64,577,10.55
2,BHMBCCMKT01,80,577,11.24
3,BHMBCCMKT01,107,577,12.17
4,BHMBCCMKT01,150,577,13.47
5,BHMBCCMKT01,177,577,15.00
6,BHMBCCMKT01,219,577,16.90
7,BHMBCCMKT01,247,577,19.04
8,BHMBCCMKT01,259,577,21.28
9,BHMBCCMKT01,266,577,23.59


    Note: the price is increasing smoothly as occupancy increases, which is a good baseline to compare future models.

# **Step 4: Demand-Based Pricing Model**

Here:

Created a demand score based on:

* Occupancy / Capacity

* QueueLength

* TrafficConditionNearby (convert to numeric: low = 1, avg = 2, high = 3)

* IsSpecialDay (already 0 or 1)

* VehicleType (we'll assign weights: car = 1, bike = 0.8, truck = 1.2)

Used below demand to adjust price from base using:

``Price = BasePrice × (1 + λ × Normalized Demand)``

In [6]:
# Step 4: Demand-Based Pricing Model

# making a copy of the original DataFrame
df_model2 = df.copy()

# mapping traffic levels to numbers
traffic_map = {'low': 1, 'average': 2, 'high': 3}
df_model2['TrafficLevelNum'] = df_model2['TrafficConditionNearby'].map(traffic_map)

# if there are unknown vehicle types, replace them with 'car' (default)
df_model2['VehicleType'] = df_model2['VehicleType'].apply(lambda x: x if x in vehicle_map else 'car')

# assigning weights to vehicle types
vehicle_map = {'car': 1.0, 'bike': 0.8, 'truck': 1.2}
df_model2['VehicleWeight'] = df_model2['VehicleType'].map(vehicle_map)

# calculating demand score using a linear function
# weights for each feature (can be adjusted later)
a = 1.0  # for occupancy rate
b = 0.5  # for queue length
c = 0.7  # for traffic level
d = 1.2  # for special day
e = 1.0  # for vehicle type

# calculating demand
df_model2['OccupancyRate'] = df_model2['Occupancy'] / df_model2['Capacity']
df_model2['DemandRaw'] = (
    a * df_model2['OccupancyRate'] +
    b * df_model2['QueueLength'] -
    c * df_model2['TrafficLevelNum'] +
    d * df_model2['IsSpecialDay'] +
    e * df_model2['VehicleWeight']
)

# normalize demand between 0 and 1
min_demand = df_model2['DemandRaw'].min()
max_demand = df_model2['DemandRaw'].max()
df_model2['DemandNormalized'] = (df_model2['DemandRaw'] - min_demand) / (max_demand - min_demand)

# price = base × (1 + λ × normalized_demand)
lambda_val = 1.0  # can be tuned
df_model2['Price_Model2'] = base_price * (1 + lambda_val * df_model2['DemandNormalized'])

# apply smooth bounding: not more than 2x or less than 0.5x base price
df_model2['Price_Model2'] = df_model2['Price_Model2'].clip(lower=0.5*base_price, upper=2*base_price)
df_model2['Price_Model2'] = df_model2['Price_Model2'].round(2)

# showing sample
print("Sample rows with demand-based pricing:")
df_model2[['SystemCodeNumber', 'OccupancyRate', 'QueueLength', 'TrafficConditionNearby', 'IsSpecialDay', 'VehicleType', 'DemandNormalized', 'Price_Model2']].head(10)


Sample rows with demand-based pricing:


,SystemCodeNumber,OccupancyRate,QueueLength,TrafficConditionNearby,IsSpecialDay,VehicleType,DemandNormalized,Price_Model2
0,BHMBCCMKT01,0.105719,1,low,0,car,0.114593,11.15
1,BHMBCCMKT01,0.110919,1,low,0,car,0.115200,11.15
2,BHMBCCMKT01,0.138648,2,low,0,car,0.176816,11.77
3,BHMBCCMKT01,0.185442,2,low,0,car,0.182280,11.82
4,BHMBCCMKT01,0.259965,2,low,0,bike,0.167629,11.68
5,BHMBCCMKT01,0.306759,3,low,0,car,0.254823,12.55
6,BHMBCCMKT01,0.379549,6,high,0,truck,0.298350,12.98
7,BHMBCCMKT01,0.428076,5,average,0,car,0.304015,13.04
8,BHMBCCMKT01,0.448873,5,average,0,car,0.306444,13.06
9,BHMBCCMKT01,0.461005,8,high,0,bike,0.377915,13.78


    Note: price increases based on demand factors like queue, occupancy, traffic, etc.

# **Step 5: Model 3 – Competitive Pricing Model**

In this model:

* Calculate distance between each parking lot using latitude & longitude

* For each record, look at nearby lots (within X km)

* Compare your lot’s price to competitors

Adjust price:

* If lot is full & nearby lots are cheaper → suggest rerouting / reduce price

* If lot is not full & competitors are expensive → you can increase your price

      Note:
      But dataset has 18,000+ rows so in for loop it will take forever to complete this code.

      So,

      * Instead of checking every row in the dataset, we’ll only check rows from other parking lots (i.e., different SystemCodeNumber), and maybe just the first N rows for demo purposes.

      * Later, for full-scale deployment, we’d batch or pre-compute proximity.

In [13]:
# recreate df_model3 from model 2 output
df_model3 = df_model2.copy()

In [14]:
# Define the haversine function again (used to calculate distance between two lat-longs)
from math import radians, sin, cos, sqrt, atan2

def haversine(lat1, lon1, lat2, lon2):
    # convert degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    # apply haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    radius_earth_km = 6371
    return radius_earth_km * c  # earth radius in km

distance_threshold = 0.5  # in kilometers


In [15]:
# for demo: limit comparisons to first 100 rows (one can increase later)
subset_df = df_model3.head(100).copy()  # process only first 100 rows for now

# create empty list to store adjusted prices
adjusted_prices = []

for i in range(len(subset_df)):
    lat1 = subset_df.loc[i, 'Latitude']
    lon1 = subset_df.loc[i, 'Longitude']
    lot_id = subset_df.loc[i, 'SystemCodeNumber']
    price = subset_df.loc[i, 'Price_Model2']
    occupancy = subset_df.loc[i, 'Occupancy']
    capacity = subset_df.loc[i, 'Capacity']
    full = occupancy >= 0.9 * capacity

    # get nearby competitors (only from other lots)
    nearby_prices = []
    for j in range(len(df_model3)):
        if df_model3.loc[j, 'SystemCodeNumber'] == lot_id:
            continue  # skip same parking lot
        lat2 = df_model3.loc[j, 'Latitude']
        lon2 = df_model3.loc[j, 'Longitude']
        dist = haversine(lat1, lon1, lat2, lon2)
        if dist <= distance_threshold:
            nearby_prices.append(df_model3.loc[j, 'Price_Model2'])

    # original price
    new_price = price

    # apply competitive pricing logic
    if nearby_prices:
        avg_comp_price = np.mean(nearby_prices)
        if full and price > avg_comp_price:
            new_price = max(base_price, avg_comp_price)
        elif not full and price < avg_comp_price:
            new_price = min(2*base_price, price + 1.0)

    adjusted_prices.append(round(new_price, 2))

# store the updated prices
subset_df['Price_Model3'] = adjusted_prices

# show result
print("Competitive pricing (on sample 100 rows):")
subset_df[['SystemCodeNumber', 'Occupancy', 'Capacity', 'Price_Model2', 'Price_Model3']].head(10)




Competitive pricing (on sample 100 rows):


,SystemCodeNumber,Occupancy,Capacity,Price_Model2,Price_Model3
0,BHMBCCMKT01,61,577,11.15,12.15
1,BHMBCCMKT01,64,577,11.15,12.15
2,BHMBCCMKT01,80,577,11.77,12.77
3,BHMBCCMKT01,107,577,11.82,12.82
4,BHMBCCMKT01,150,577,11.68,12.68
5,BHMBCCMKT01,177,577,12.55,13.55
6,BHMBCCMKT01,219,577,12.98,13.98
7,BHMBCCMKT01,247,577,13.04,14.04
8,BHMBCCMKT01,259,577,13.06,14.06
9,BHMBCCMKT01,266,577,13.78,13.78


    Note: Prices are slightly increased if nearby competitors are charging more



# **Step 6 – Real-Time Simulation using Pathway**

* it simulates real-time data flow by using Pathway to simulate streaming input

* This is where simulation of the data is done which is coming in one row at a time (like a real-time stream), and apply pricing model to each row as it comes in.


Step-by-Step Plan:

* Install Pathway

* Combine LastUpdatedDate and LastUpdatedTime into a full timestamp

* Applied Model 2: Demand-Based Pricing

* Sorted it by Timestamp to simulate streaming behavior

* Shown how prices vary across locations over time

In [16]:
df.head()


,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00


**Step 6.1: Install Pathway**

In [17]:
# Installing Pathway (only needs to be done once)
!pip install -U pathway

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 14.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of boto3 to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 77.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 

**Step 6.2: Create Timestamp Column**

In [18]:
# Step 6.2: Creating timestamp from date and time columns

# make a new column that merges both columns into one timestamp
# create timestamp from date and time, telling pandas that day comes first
df_model3['Timestamp'] = pd.to_datetime(df_model3['LastUpdatedDate'] + ' ' + df_model3['LastUpdatedTime'], dayfirst=True)


# check if it looks fine
df_model3[['LastUpdatedDate', 'LastUpdatedTime', 'Timestamp']].head()


,LastUpdatedDate,LastUpdatedTime,Timestamp
0,04-10-2016,07:59:00,2016-10-04 07:59:00
1,04-10-2016,08:25:00,2016-10-04 08:25:00
2,04-10-2016,08:59:00,2016-10-04 08:59:00
3,04-10-2016,09:32:00,2016-10-04 09:32:00
4,04-10-2016,09:59:00,2016-10-04 09:59:00


**Step 6.3: Real-Time Simulation with Pathway**

In [26]:
# Step 6: Real-Time Simulation (Offline Version)

# sort the data by timestamp to simulate real-time arrival
df_simulated = df_model3.sort_values(by='Timestamp').copy()

# we already have price columns like:
# - df_simulated['Price_Model2'] → demand-based pricing
# - df_simulated['Price_Model3'] → competitive pricing

# this sorted dataframe will now be used in Step 7 for plotting
df_simulated[['SystemCodeNumber', 'Timestamp', 'Price_Model2']].head()

,SystemCodeNumber,Timestamp,Price_Model2
0,BHMBCCMKT01,2016-10-04 07:59:00,11.15
5248,BHMNCPHST01,2016-10-04 07:59:00,11.60
3936,BHMMBMMBX01,2016-10-04 07:59:00,12.05
6560,BHMNCPNST01,2016-10-04 07:59:00,12.21
17056,Shopping,2016-10-04 07:59:00,11.98


    Note:

    * Multiple parking lots are reporting data for the same timestamp (07:59:00)

    * Each parking lot has a different Price_Model2 depending on its demand inputs

# **Step 7: Bokeh Visualization – Dynamic Pricing Over Time**

* Bokeh to create an interactive line chart showing how Model 2 prices change over time for different parking lots.

What this chart will do:
* X-axis: Time (``Timestamp``)

* Y-axis: Model 2 Price (``Price_Model2``)

* Each colored line: A different ``SystemCodeNumber`` (i.e. per parking lot)

* One can zoom, pan, and hover for exact values

**Step 7.1: Install Bokeh**

In [27]:
!pip install bokeh

**Step 7.2: Plot Prices Using Bokeh**

In [28]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, Legend
from bokeh.palettes import Category10
from bokeh.layouts import layout
import pandas as pd

# activate Bokeh inside notebook
output_notebook()

# pick 5 unique parking lots to plot
selected_lots = df_simulated['SystemCodeNumber'].unique()[:5]

# create figure
p = figure(x_axis_type="datetime", width=900, height=400,
           title="Model 2 Price Over Time (Demand-Based Pricing)",
           tools="pan,wheel_zoom,box_zoom,reset,hover,save")

# assign colors
colors = Category10[10]

legend_items = []

for i, lot in enumerate(selected_lots):
    lot_df = df_simulated[df_simulated['SystemCodeNumber'] == lot]
    lot_df = lot_df.sort_values('Timestamp')

    line = p.line(x=lot_df['Timestamp'], y=lot_df['Price_Model2'],
                  line_width=2, color=colors[i % len(colors)],
                  legend_label=lot, name=lot)

# format hover
p.hover.tooltips = [
    ("Parking Lot", "$name"),
    ("Time", "@x{%F %T}"),
    ("Price", "@y")
]
p.hover.formatters = {"@x": "datetime"}
p.legend.location = "top_left"
p.xaxis.axis_label = "Timestamp"
p.yaxis.axis_label = "Price (Model 2)"

# show plot
show(p)


    Insight from the chart:

    1. The "Shopping" parking lot has frequent price spikes — probably due to high queue length or special day effects.

    2. Others show smoother trends, reflecting stable traffic or demand.